<a href="https://colab.research.google.com/github/KJM94/pym/blob/main/src/2021%EB%85%84_4%EC%9B%94_19%EC%9D%BC_%EB%B9%85%EC%BF%BC%EB%A6%AC%EC%99%80_%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EC%97%B0%EB%8F%99_%EA%B6%8C%EC%A0%95%EB%AF%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 개요
- 교과목명: 빅데이터 분석/분석용 데이터 구축
- 평가일: 2021년 4월 19일
- 평가자: 권정민

## 캐글 데이터 다운로드
- 캐글 데이터를 다운로드 받습니다. 

In [1]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle
!echo '{"username":"kwonjungmin","key":"71b01b2de87cdc8cf37537ed0b9057b2"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c tabular-playground-series-apr-2021

  0% 0.00/2.13M [00:00<?, ?B/s]
100% 2.13M/2.13M [00:00<00:00, 70.9MB/s]
  0% 0.00/2.07M [00:00<?, ?B/s]
100% 2.07M/2.07M [00:00<00:00, 111MB/s]
  0% 0.00/879k [00:00<?, ?B/s]
100% 879k/879k [00:00<00:00, 57.9MB/s]


In [4]:
!ls

sample_data  sample_submission.csv  test.csv.zip  train.csv.zip


In [5]:
!unzip "*.zip"

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  train.csv.zip
  inflating: train.csv               

2 archives were successfully processed.


## 사용자 계정 인증

In [6]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## 빅쿼리 사용 예제

In [8]:
%%bigquery --project kminproject
SELECT 
  COUNT(*) as total_rows
FROM `bigquery-public-data.samples.gsod`

,total_rows
0,114420316


In [9]:
!ls

adc.json     sample_submission.csv  test.csv.zip  train.csv.zip
sample_data  test.csv		    train.csv


### 빅쿼리 파이썬 연동


In [10]:
!pip install --upgrade pandas-gbq 'google-cloud-bigquery[bqstorage,pandas]'

     |████████████████████████████████| 225kB 21.6MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 153kB 37.8MB/s 
     |████████████████████████████████| 522kB 33.1MB/s 
     |████████████████████████████████| 645kB 22.9MB/s 
ERROR: google-cloud-storage 1.18.1 has requirement google-resumable-media<0.5.0dev,>=0.3.1, but you'll have google-resumable-media 1.2.0 which is incompatible.
  Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Found existing installation: google-cloud-bigquery-storage 1.1.0
    Un

In [11]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas

project_id = 'kminproject'
sql = """
  SELECT 
    COUNT(*) as total_rows
  FROM [bigquery-public-data.samples.gsod]
"""

df = pandas.read_gbq(sql, project_id=project_id, dialect='legacy')
df.head()

,total_rows
0,114420316


### 데이터 적재하기
- 빅쿼리에서 데이터세트 만들기

In [12]:
import pandas as pd

BASE_DIR = "./"
train = pd.read_csv(BASE_DIR + 'train.csv')
test = pd.read_csv(BASE_DIR + 'test.csv')

train.shape, test.shape

((100000, 12), (100000, 11))

In [14]:
project_id = 'kminproject'

train_table_id = 'kaggle_titanic.train'
train.to_gbq(train_table_id,  project_id = project_id, if_exists = "replace")
test_table_id = 'kaggle_titanic.test'
test.to_gbq(test_table_id, project_id = project_id, if_exists = "replace")

1it [00:08,  8.27s/it]
1it [00:06,  6.18s/it]


### 데이터 불러오기

In [15]:
project_id = 'kminproject'
train_sql = """
  SELECT 
    *
  FROM [kminproject.kaggle_titanic.train]
"""

test_sql = """
  SELECT 
    *
  FROM [kminproject.kaggle_titanic.test]
"""

train = pandas.read_gbq(train_sql, project_id=project_id, dialect='legacy')
test = pandas.read_gbq(test_sql, project_id=project_id, dialect='legacy')

train.shape, test.shape

((100000, 12), (100000, 11))